In [2]:
#!/usr/bin/env python
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression


from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

import warnings

from scipy.optimize.linesearch import LineSearchWarning

from helper_functions import evaluate, plot_roc, plot_confusion_matrix,plot_precision_recall

ModuleNotFoundError: No module named 'helper_functions'

In [ ]:
@ignore_warnings(category=ConvergenceWarning)
@ignore_warnings(category=LineSearchWarning)
def train_model(ratio,solver,max_iters, c, fit_intercept):
    # Load training and testing data
    X_train = np.loadtxt('X_train.csv', delimiter=',', skiprows=1) # each row is an observation with 6 features
    y_train = np.loadtxt('y_train.csv', delimiter=',', skiprows=1)[:,1] # each row corresponds to actual value of observation from ^

    # Set up the model
    classification = LogisticRegression(solver=solver, max_iter=max_iters,fit_intercept=fit_intercept,C=c)

    # Partition data into train and test
    train_x, cross_validation_x_data, train_y, cross_validation_actual_y_values = train_test_split(X_train, y_train, test_size=(1-ratio), random_state=0)

    # Train model
    classification.fit(train_x, train_y)

    # Test model
    predicted_validation_y = classification.predict(cross_validation_x_data)
    # predict probabilities
    predicted_validation_y_probs = classification.predict_proba(cross_validation_x_data)
    
    
    
    return classification, predicted_validation_y, cross_validation_actual_y_values,predicted_validation_y_probs


title = "Logistical Regression"

ratio = 0.83
solver = 'newton-cg'
max_iter = 70
c = 104
fit_intercept = False




trained_classification, predicted_validation_y, validation_actual_y_values,predicted_validation_y_probs = train_model(ratio, solver, max_iter, c, fit_intercept)

accuracy, sensitivity,con_matrix = evaluate(predicted_validation_y, validation_actual_y_values)

X_test = np.loadtxt('X_test.csv', delimiter=',', skiprows=1)
y_pred = trained_classification.predict(X_test)

In [ ]:
plot_roc(validation_actual_y_values,predicted_validation_y_probs[:, 1], title)

In [ ]:
# what the lectures call ROC curve
plot_precision_recall(validation_actual_y_values,predicted_validation_y_probs[:, 0],title)

In [ ]:

plot_confusion_matrix(con_matrix,["Epithelial", "Stromal"],title,text_x=-0.7,text_y=-1.1)

In [ ]:
# Arrange answer in two columns. First column (with header "Id") is an
# enumeration from 0 to n-1, where n is the number of test points. Second
# column (with header "EpiOrStroma" is the predictions.
test_header = "Id,EpiOrStroma"
n_points = X_test.shape[0]
y_pred_pp = np.ones((n_points, 2))
y_pred_pp[:, 0] = range(n_points)
y_pred_pp[:, 1] = y_pred
np.savetxt('my_submission.csv', y_pred_pp, fmt='%d', delimiter=",",
           header=test_header, comments="")

# Note: fmt='%d' denotes that all values should be formatted as integers which
# is appropriate for classification. For regression, where the second column
# should be floating point, use fmt='%d,%f'.
    